In [9]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go 
import yfinance as yf


tickers = ['AAPL' , 'GOOGL', 'BRK-B' , 'PFE.DE','GC=F'] #definimos os tickers das ações 
## Ações 
prices = yf.download(tickers=tickers , start = '2017-01-01', end='2020-01-01')['Adj Close']
prices.dropna(inplace=True)

returns = (prices/prices.shift(1) -1)
returns.dropna(inplace=True)
ativos = returns.columns.to_list()
returns


[*********************100%%**********************]  5 of 5 completed


Ticker,AAPL,BRK-B,GC=F,GOOGL,PFE.DE
Date,,,,,
2017-01-04,-0.001119,0.001526,0.002930,-0.000297,0.000474
2017-01-05,0.005085,-0.004754,0.013662,0.006499,-0.004895
2017-01-06,0.011148,0.000674,-0.006612,0.014994,0.007299
2017-01-09,0.009160,-0.008506,0.009898,0.002387,-0.001260
2017-01-10,0.001008,-0.003395,0.000591,-0.001414,-0.004259
...,...,...,...,...,...
2019-12-19,0.001001,0.000889,0.003803,0.003351,0.005747
2019-12-20,-0.002071,0.004884,-0.002368,-0.003848,0.020000
2019-12-23,0.016318,-0.003668,0.005289,-0.000437,-0.008403


In [10]:
### starting portfolio simulation
number_sim = 3000
weight = np.zeros((number_sim, len(tickers)))
Sigma = returns.cov() # matriz de covariancia
mean_returns = returns.mean()

expected_return = np.zeros(number_sim)
expected_risk = np.zeros(number_sim)
sharpe_ratio  = np.zeros(number_sim)





for k in range(number_sim):
    # gerar pesos aleatórios e normalizados
    w = np.array(np.random.random(len(tickers)))
    w= w/np.sum(w)
    weight[k,:]= w 

    #retorno log esperado:
    expected_return[k] = np.sum( mean_returns *w) *252 # anualizado

    # risco: 
    #expected_risk  = w.T @ Sigma @ w
    expected_risk[k]  = np.sqrt(np.dot(w.T, np.dot(Sigma, w)) * 252) 

    sharpe_ratio[k]   = expected_return[k]/ expected_risk[k]


In [11]:
import numpy as np
import pandas as pd

# Assumindo que 'returns' é o DataFrame com os retornos diários das ações
# e que 'returns' já foi carregado anteriormente

number_sim = 1000
num_assets = len(tickers)  # Número de ativos
weight = np.zeros((number_sim, num_assets))
Sigma = returns.cov() # matriz de covariancia
mean_returns = returns.mean()

expected_return = np.zeros(number_sim)
expected_risk = np.zeros(number_sim)
sharpe_ratio = np.zeros(number_sim)

for k in range(number_sim):
    # Gerar pesos aleatórios e normalizados
    w = np.random.random(num_assets)
    w = w / np.sum(w)
    weight[k, :] = w 

    # Retorno log esperado (anualizado)
    expected_return[k] = np.sum(mean_returns * w) * 252

    # Risco (anualizado)
    expected_risk[k] = np.sqrt(np.dot(w.T, np.dot(Sigma, w)) * 252)

    # Índice de Sharpe
    sharpe_ratio[k] = (expected_return[k] / expected_risk[k]) 

# Criar DataFrame
portfolio_df = pd.DataFrame(weight, columns=[f'{tickers[i]}' for i in range(num_assets)])
portfolio_df['Expected Return(%)'] = expected_return*100
portfolio_df['Risk(%)'] = expected_risk *100
portfolio_df['Sharpe Ratio'] = sharpe_ratio



portfolio_df


,AAPL,GOOGL,BRK-B,PFE.DE,GC=F,Expected Return(%),Risk(%),Sharpe Ratio
0,0.293615,0.221730,0.333244,0.069658,0.081753,18.791010,11.139459,1.686887
1,0.208570,0.227112,0.101757,0.083599,0.378963,16.658172,12.170946,1.368683
2,0.088134,0.245546,0.160246,0.406266,0.099807,16.879047,13.737668,1.228669
3,0.403076,0.106010,0.232746,0.094317,0.163852,21.619618,13.029576,1.659273
4,0.320809,0.208299,0.065410,0.191850,0.213632,20.713894,14.390577,1.439407
...,...,...,...,...,...,...,...,...
995,0.233695,0.276233,0.300415,0.020866,0.168791,16.837759,10.238026,1.644629
996,0.270233,0.245712,0.160884,0.259058,0.064113,20.187882,14.121403,1.429595
997,0.174797,0.082999,0.540312,0.182545,0.019345,16.445315,9.868163,1.666502
998,0.270229,0.280837,0.008956,0.139617,0.300362,19.020757,14.174637,1.341887


In [12]:
import plotly.express as px

# Encontrar a linha com o máximo índice de Sharpe
max_sharpe_idx = portfolio_df['Sharpe Ratio'].idxmax()
max_sharpe_portfolio = portfolio_df.loc[max_sharpe_idx]

# Encontrar a linha com a mínima variância (risco)
min_variance_idx = portfolio_df['Risk(%)'].idxmin()
min_variance_portfolio = portfolio_df.loc[min_variance_idx]

# Automatizar a criação do dicionário hover_data
hover_data = {col: ':.4f' for col in portfolio_df.columns}

# Criar gráfico de dispersão com Plotly, incluindo os pesos no hover_data
fig = px.scatter(
    portfolio_df, 
    x='Risk(%)', 
    y='Expected Return(%)', 
    color='Sharpe Ratio',
    title='Risco vs Retorno com Índice de Sharpe por  Monte Carlo',
    labels={'Risk(%)': 'Risco (%)', 'Expected Return(%)': 'Retorno (%)'},
    color_continuous_scale=px.colors.sequential.Bluered,
    hover_data=hover_data
)

# Adicionar contorno aos marcadores de dispersão
fig.update_traces(marker=dict(line=dict(width=2, color='black')))

# Adicionar marcadores para o portfólio com máximo Sharpe e mínima variância
fig.add_scatter(
    x=[max_sharpe_portfolio['Risk(%)']], 
    y=[max_sharpe_portfolio['Expected Return(%)']], 
    mode='markers+text', 
    marker=dict(color='green', size=12, symbol='star', line=dict(color='black', width=1)), 
    name='Máximo Sharpe',
)

fig.add_scatter(
    x=[min_variance_portfolio['Risk(%)']], 
    y=[min_variance_portfolio['Expected Return(%)']], 
    mode='markers+text', 
    marker=dict(color='blue', size=12, symbol='star', line=dict(color='black', width=1)), 
    name='Mínima Variância'
)

# Atualizar layout para mover a legenda para a parte inferior
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.3,
        xanchor="center",
        x=0.5
    )
)

# Mostrar gráfico
fig.show()


In [13]:
correlation_matrix = round(returns.corr(), 3)
# Mascarar a matriz de correlação para obter apenas a triangular inferior
mask = np.tril(np.ones_like(correlation_matrix, dtype=bool))
correlation_matrix_masked = correlation_matrix.mask(~mask)

# Plotar a matriz de correlação usando Plotly
fig = px.imshow(
    correlation_matrix_masked, 
    text_auto=True, 
    aspect="equal", 
    color_continuous_scale='Blues',  # Escala de cor azul
    title="Matriz de Correlação"
)

# Melhorar o aspecto do gráfico
fig.update_layout(
    width=600,  # Largura da figura
    height=600,  # Altura da figura
    margin=dict(l=100, r=50, t=50, b=50),  # Margens
    font=dict(size=14),  # Tamanho da fonte
    plot_bgcolor='white',  # Cor de fundo branca
    paper_bgcolor='white'  # Cor de fundo branca
)

fig.show()

### Vamos analisar em termos de retornos como a carteira se sairia daqui 

In [14]:
min_variance_portfolio

AAPL                   0.005800
GOOGL                  0.188461
BRK-B                  0.561816
PFE.DE                 0.048268
GC=F                   0.195656
Expected Return(%)    10.850758
Risk(%)                7.424205
Sharpe Ratio           1.461538
Name: 242, dtype: float64

In [15]:
max_sharpe_portfolio

AAPL                   0.268464
GOOGL                  0.004732
BRK-B                  0.495046
PFE.DE                 0.026627
GC=F                   0.205131
Expected Return(%)    17.084218
Risk(%)                9.093450
Sharpe Ratio           1.878739
Name: 372, dtype: float64

In [16]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go

tickers.append('SPY')

# Criar o dicionário de pesos automaticamente a partir dos tickers
weights = {ticker: max_sharpe_portfolio[ticker] for ticker in tickers if ticker in max_sharpe_portfolio.index}

# Baixar os preços ajustados das ações e do índice SPY
prices = yf.download(tickers=tickers, start='2020-01-01', end='2022-01-01')['Adj Close']
prices.dropna(inplace=True)

# Calcular os retornos diários
returns = prices.pct_change().dropna()

# Calcular o retorno diário do portfólio
portfolio_returns = returns[list(weights.keys())].mul(list(weights.values()), axis=1).sum(axis=1)

# Calcular o retorno acumulado do portfóliocumprod(
cumulative_portfolio_returns = ((1 + portfolio_returns).cumprod() - 1)*100

# Calcular o retorno acumulado do SPY
cumulative_spy_returns = ((1 + returns['SPY']).cumprod() - 1)*100

# Plotar os retornos acumulados
fig = go.Figure()

fig.add_trace(go.Scatter(x=cumulative_portfolio_returns.index, y=cumulative_portfolio_returns, 
                         mode='lines', name='Portfólio', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=cumulative_spy_returns.index, y=cumulative_spy_returns, 
                         mode='lines', name='SPY', line=dict(color='red')))

fig.update_layout(title='Retorno Acumulado do Portfólio vs SPY',
                  xaxis_title='Data',
                  yaxis_title='Retorno Acumulado(%)',
                  template='plotly_white')

fig.show()

[*********************100%%**********************]  6 of 6 completed
